## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values....
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.0.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1029, 2445, 701, 1995, 4650, 2474, 873, 4781, 888, 2511],
 [3464, 262, 1610, 1624, 4632, 960, 2920],
 [2812, 4742, 4932, 4073],
 [4211, 1104, 1068, 2889, 1198, 441],
 [4181, 4632, 3826, 1113, 1645, 4368, 4632, 3229, 375, 4637],
 [1012,
  4785,
  441,
  1102,
  3250,
  801,
  3581,
  2561,
  211,
  510,
  533,
  4356,
  3917,
  1963,
  2920],
 [3297, 2393, 161, 200, 2105, 876, 2185, 161, 1368, 1308, 582],
 [59, 4301, 2322, 3167, 1727, 3804, 801, 3033, 1368, 1308, 582],
 [1785, 535, 4061, 4525, 633, 1608, 2551, 1843, 801, 254],
 [4129, 2047, 1806, 634, 4970, 2563, 4622, 245],
 [870, 4680, 4319, 1846, 713, 1445, 2566, 490, 4542, 2455, 2793],
 [2889, 1019, 4650, 1608, 801, 1727],
 [3966, 382, 280, 2680, 4122, 3230, 1561, 638, 4650],
 [2818, 3143, 2182, 2810, 3941, 623, 4005, 1368, 1308, 582],
 [223, 2456, 4105, 2370, 3353, 1368, 1308, 582],
 [1723, 2109, 16, 4120, 804, 917, 1634, 897, 4647, 1277],
 [4802, 4719, 262],
 [3127, 2252, 39, 127, 801, 3342, 1571, 2920],
 [395, 2842, 1610, 2283,

### Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4781  888 2511]
 [   0    0    0 ... 4632  960 2920]
 [   0    0    0 ... 4742 4932 4073]
 ...
 [   0    0    0 ... 1368 1308  582]
 [   0    0    0 ... 1079 2553  260]
 [   0    0    0 ...   70 3766 2846]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1029,
       2445,  701, 1995, 4650, 2474,  873, 4781,  888, 2511])

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 13s 1ms/sample - loss: 0.3352 - accuracy: 0.8416 - val_loss: 0.1940 - val_accuracy: 0.9191
Epoch 2/10
12250/12250 [==============================] - 11s 876us/sample - loss: 0.1337 - accuracy: 0.9485 - val_loss: 0.1866 - val_accuracy: 0.9253
Epoch 3/10
12250/12250 [==============================] - 11s 873us/sample - loss: 0.0905 - accuracy: 0.9669 - val_loss: 0.2189 - val_accuracy: 0.9226
Epoch 4/10
12250/12250 [==============================] - 11s 875us/sample - loss: 0.0599 - accuracy: 0.9789 - val_loss: 0.2387 - val_accuracy: 0.9163
Epoch 5/10
12250/12250 [==============================] - 11s 878us/sample - loss: 0.0318 - accuracy: 0.9912 - val_loss: 0.3693 - val_accuracy: 0.9180
Epoch 6/10
12250/12250 [==============================] - 11s 879us/sample - loss: 0.0218 - accuracy: 0.9933 - val_loss: 0.2839 - val_accuracy: 0.9178
Epoch 7/10
12250/12250 [=======================

### Adding Dropout 

In [35]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 13s 1ms/sample - loss: 0.3391 - accuracy: 0.8457 - val_loss: 0.2025 - val_accuracy: 0.9145
Epoch 2/10
12250/12250 [==============================] - 11s 885us/sample - loss: 0.1489 - accuracy: 0.9426 - val_loss: 0.1933 - val_accuracy: 0.9190
Epoch 3/10
12250/12250 [==============================] - 11s 925us/sample - loss: 0.1091 - accuracy: 0.9581 - val_loss: 0.2010 - val_accuracy: 0.9206
Epoch 4/10
12250/12250 [==============================] - 12s 941us/sample - loss: 0.0814 - accuracy: 0.9702 - val_loss: 0.2366 - val_accuracy: 0.9175
Epoch 5/10
12250/12250 [==============================] - 12s 946us/sample - loss: 0.0613 - accuracy: 0.9793 - val_loss: 0.2704 - val_accuracy: 0.9143
Epoch 6/10
12250/12250 [==============================] - 11s 917us/sample - loss: 0.0435 - accuracy: 0.9852 - val_loss: 0.3130 - val_accuracy: 0.9160
Epoch 7/10
12250/12250 [=======================

### Performance Metrics And Accuracy

In [37]:
y_pred=model.predict_classes(X_test)

In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(y_test,y_pred)

array([[3149,  270],
       [ 289, 2327]], dtype=int64)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9073736536868269